In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# Загружаем модель и токенизатор, кладем их на GPU
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
texts = [
    "Как привлечь к себе благосостояние?",
    "Че делать когда мучает совесть после пьянки?",
    "Почему вы хотите знать , что будет завтра?",
]

In [4]:
# Подавать просто так входы в модель нельзя. Различные instruct-LLM обучены на различных форматах 
# и нужно обязательно ими пользоваться. Если этого не делать, то качество генераций может как снизиться, так 
# и стать совсем бессмысленным. Пример форматирования ниже!
print(tokenizer.apply_chat_template(
    [
        {"role": "system", "content": "Дай ответ в стиле mail.ru ответов"}, 
        {"role": "user", "content": texts[0]}
    ], 
    tokenize=False,
    add_generation_prompt=True,
))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Дай ответ в стиле mail.ru ответов<|eot_id|><|start_header_id|>user<|end_header_id|>

Как привлечь к себе благосостояние?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [7]:
for text_idx, text in  enumerate(texts):
    print("\n\n")
    print(f"-----Текст {text_idx}-----")
    print(text)
    # токенизируем текст, возаращаем тензоры для pytorch
    input_ids = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "Дай ответ в стиле mail.ru ответов"}, 
            {"role": "user", "content": text}
        ], 
        return_tensors="pt",
        add_generation_prompt=True
    )
    # кладем тензоры на то же устройство (gpu, cpu), что и модель
    input_ids = input_ids.to(device) 
    
    # генерируем токены с сэмплингом и температурой, о том, как это работает, можно узнать на курсе!
    # также некоторые технические аргументы тоже опустим, но если очень хочется, то обо всех можно почитать в документации
    # https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationConfig
    for i in range(1):
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            do_sample=True,
            temperature=0.8,
            max_length=200,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=True,
        )
        # проводим детокенизацию, т.е. превращаем сгенерированные токены обратно в текст
        generated_token_ids = outputs[0][input_ids.size(1):]
        candidate = tokenizer.decode(generated_token_ids)
        print(f"\n-----Кандидат {i}-----")
        print(candidate)




-----Текст 0-----
Как привлечь к себе благосостояние?

-----Кандидат 0-----
Благосостояние - это чувство комфорта и безопасности, которое можно привлечь в свою жизнь посредством изменений в образе жизни и мышлении.

Чтобы привлечь к себе благосостояние, можно попробовать следующие шаги:

1. **Измените свою энергетику**: Обращайте внимание на свои мысли, чувства и действия. Когда вы чувствуете себя счастливым, оптимистичным и позитивным, ваша энергия становится привлекательной для благополучия.
2. **Практикуйте самообеспечение**: Заботьтесь о своем физическом и эмоциональном здоровье. Включайте в свой распорядок дня упражнения, правильное



-----Текст 1-----
Че делать когда мучает совесть после пьянки?

-----Кандидат 0-----
Если мучает совесть после пьянки, то есть несколько вариантов, чтобы решить эту проблему:

1. **Признай свои ошибки**: Начни с признания того, что ты совершил ошибку, и извлеки из нее урок. Понимай, что пьянка не решает проблем, а только усугубляет их.

2. **Прос